# Entrenamiento de Modelo con Nesterov Accelerated Gradient (NAG)

Este notebook implementa un modelo de regresión usando **Nesterov Accelerated Gradient (NAG)** para predecir valores basados en datos de series temporales del archivo `dryer.dat`.

## ¿Qué es Nesterov Accelerated Gradient?

NAG es una mejora sobre el Momentum estándar que **anticipa** la dirección del gradiente antes de calcularlo. Es como "mirar hacia adelante" antes de dar el paso.

### Diferencia clave con SGD estándar:
- **SGD:** Calcula el gradiente en la posición actual
- **Momentum:** Acumula velocidad de gradientes pasados
- **Nesterov:** Calcula el gradiente en la posición anticipada (más inteligente)

## División de datos:
- **70%** - Entrenamiento (Train)
- **20%** - Validación (Valid)
- **10%** - Prueba (Test)

**Importante**: Los datos se dividen de forma secuencial para respetar el orden temporal.

## 1. Importar Librerías

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

## 2. Cargar y Explorar los Datos

In [2]:
# Cargar datos desde dryer.dat
print("Cargando datos desde dryer.dat...")
data = np.loadtxt('dryer.dat')

# Separar características (X) y variable objetivo (y)
X = data[:, 0].reshape(-1, 1)  # Primera columna como entrada
y = data[:, 1].reshape(-1, 1)  # Segunda columna como salida

print(f"Datos cargados: {len(X)} muestras")
print(f"Forma de X: {X.shape}")
print(f"Forma de y: {y.shape}")

Cargando datos desde dryer.dat...
Datos cargados: 1000 muestras
Forma de X: (1000, 1)
Forma de y: (1000, 1)


## 3. División Secuencial de los Datos

Como son datos de series temporales, NO mezclamos los datos. Mantenemos el orden temporal.

In [3]:
# División de datos SECUENCIAL (para series temporales): 70% train, 20% valid, 10% test
# No mezclamos los datos, mantenemos el orden temporal
n_samples = len(X)
train_end = int(0.7 * n_samples)
valid_end = int(0.9 * n_samples)

# División secuencial
X_train = X[:train_end]
y_train = y[:train_end]

X_valid = X[train_end:valid_end]
y_valid = y[train_end:valid_end]

X_test = X[valid_end:]
y_test = y[valid_end:]

print(f"\nDivisión de datos SECUENCIAL (series temporales):")
print(f"Train: {len(X_train)} muestras ({len(X_train)/len(X)*100:.1f}%) - Índices [0:{train_end}]")
print(f"Valid: {len(X_valid)} muestras ({len(X_valid)/len(X)*100:.1f}%) - Índices [{train_end}:{valid_end}]")
print(f"Test:  {len(X_test)} muestras ({len(X_test)/len(X)*100:.1f}%) - Índices [{valid_end}:{n_samples}]")


División de datos SECUENCIAL (series temporales):
Train: 700 muestras (70.0%) - Índices [0:700]
Valid: 200 muestras (20.0%) - Índices [700:900]
Test:  100 muestras (10.0%) - Índices [900:1000]


## 4. Normalización/Estandarización de los Datos

In [4]:
# Normalizar datos usando StandardScaler
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_train_norm = scaler_X.fit_transform(X_train)
X_valid_norm = scaler_X.transform(X_valid)
X_test_norm = scaler_X.transform(X_test)

y_train_norm = scaler_y.fit_transform(y_train)
y_valid_norm = scaler_y.transform(y_valid)
y_test_norm = scaler_y.transform(y_test)

print("Datos normalizados exitosamente")

Datos normalizados exitosamente


## 5. Implementación del Modelo con Nesterov Accelerated Gradient

Implementamos desde cero un modelo de regresión usando **Nesterov Accelerated Gradient (NAG)**.

### Fórmula de Nesterov:
```
v_t = momentum * v_{t-1} + learning_rate * gradiente(θ - momentum * v_{t-1})
θ_t = θ_{t-1} - v_t
```

Donde:
- `v_t` es la velocidad (acumulación de gradientes)
- `momentum` es típicamente 0.9
- El gradiente se calcula en una posición **anticipada**: `θ - momentum * v_{t-1}`

In [5]:
# Implementación de Nesterov Accelerated Gradient (NAG)
class NesterovRegressor:
    def __init__(self, learning_rate=0.01, n_epochs=700, batch_size=32, momentum=0.9):
        self.learning_rate = learning_rate
        self.n_epochs = n_epochs
        self.batch_size = batch_size
        self.momentum = momentum  # Factor de momentum (típicamente 0.9)
        self.weights = None
        self.bias = None
        self.velocity_w = None  # Velocidad para los pesos
        self.velocity_b = None  # Velocidad para el bias
        self.train_losses = []
        self.valid_losses = []
        
    def _initialize_parameters(self, n_features):
        """Inicializar pesos, bias y velocidades"""
        self.weights = np.random.randn(n_features, 1) * 0.01
        self.bias = np.zeros((1, 1))
        # Inicializar velocidades en cero
        self.velocity_w = np.zeros((n_features, 1))
        self.velocity_b = np.zeros((1, 1))
    
    def _compute_loss(self, y_true, y_pred):
        """Calcular MSE (Mean Squared Error)"""
        return np.mean((y_true - y_pred) ** 2)
    
    def _forward(self, X, weights=None, bias=None):
        """Propagación hacia adelante
        
        Args:
            X: datos de entrada
            weights: pesos a usar (si es None, usa self.weights)
            bias: bias a usar (si es None, usa self.bias)
        """
        if weights is None:
            weights = self.weights
        if bias is None:
            bias = self.bias
        return np.dot(X, weights) + bias
    
    def _backward(self, X, y_true, y_pred):
        """Propagación hacia atrás (calcular gradientes)"""
        n_samples = X.shape[0]
        
        # Gradientes
        dw = (-2/n_samples) * np.dot(X.T, (y_true - y_pred))
        db = (-2/n_samples) * np.sum(y_true - y_pred)
        
        return dw, db
    
    def fit(self, X_train, y_train, X_valid=None, y_valid=None):
        """Entrenar el modelo usando Nesterov Accelerated Gradient"""
        n_samples, n_features = X_train.shape
        self._initialize_parameters(n_features)
        
        # Entrenar por épocas
        for epoch in range(self.n_epochs):
            X = X_train
            Y = y_train
            
            # Mini-batch NAG
            for i in range(0, n_samples, self.batch_size):
                X_batch = X[i:i+self.batch_size]
                y_batch = Y[i:i+self.batch_size]
                
                # ===== PASO 1: Calcular posición anticipada (lookahead) =====
                # Esta es la clave de Nesterov: calcular gradiente en la posición futura
                weights_lookahead = self.weights - self.momentum * self.velocity_w
                bias_lookahead = self.bias - self.momentum * self.velocity_b
                
                # ===== PASO 2: Forward pass en posición anticipada =====
                y_pred = self._forward(X_batch, weights_lookahead, bias_lookahead)
                
                # ===== PASO 3: Backward pass (gradientes en posición anticipada) =====
                dw, db = self._backward(X_batch, y_batch, y_pred)
                
                # ===== PASO 4: Actualizar velocidades =====
                self.velocity_w = self.momentum * self.velocity_w + self.learning_rate * dw
                self.velocity_b = self.momentum * self.velocity_b + self.learning_rate * db
                
                # ===== PASO 5: Actualizar parámetros usando las velocidades =====
                self.weights -= self.velocity_w
                self.bias -= self.velocity_b
            
            # Calcular pérdida en train
            y_train_pred = self._forward(X_train)
            train_loss = self._compute_loss(y_train, y_train_pred)
            self.train_losses.append(train_loss)
            
            # Calcular pérdida en validación (si se proporciona)
            if X_valid is not None and y_valid is not None:
                y_valid_pred = self._forward(X_valid)
                valid_loss = self._compute_loss(y_valid, y_valid_pred)
                self.valid_losses.append(valid_loss)
                
                if (epoch + 1) % 10 == 0:
                    print(f"Época {epoch+1}/{self.n_epochs} - Train Loss: {train_loss:.6f} - Valid Loss: {valid_loss:.6f}")
            else:
                if (epoch + 1) % 10 == 0:
                    print(f"Época {epoch+1}/{self.n_epochs} - Train Loss: {train_loss:.6f}")
    
    def predict(self, X):
        """Hacer predicciones"""
        return self._forward(X)
    
    def evaluate(self, X, y):
        """Evaluar el modelo"""
        y_pred = self.predict(X)
        mse = self._compute_loss(y, y_pred)
        rmse = np.sqrt(mse)
        
        # R² score
        ss_tot = np.sum((y - np.mean(y)) ** 2)
        ss_res = np.sum((y - y_pred) ** 2)
        r2 = 1 - (ss_res / ss_tot)
        
        return {'MSE': mse, 'RMSE': rmse, 'R2': r2}

print("Clase NesterovRegressor definida exitosamente")

Clase NesterovRegressor definida exitosamente


## 6. Entrenar el Modelo con Nesterov

### Hiperparámetros:
- **learning_rate**: 0.01 (tasa de aprendizaje)
- **n_epochs**: 200 (número de épocas)
- **batch_size**: 32 (tamaño del mini-batch)
- **momentum**: 0.9 (factor de aceleración de Nesterov)

In [6]:
# Crear y entrenar el modelo con Nesterov
print("="*60)
print("ENTRENAMIENTO DEL MODELO NESTEROV ACCELERATED GRADIENT")
print("="*60)

model = NesterovRegressor(learning_rate=0.01, n_epochs=200, batch_size=32, momentum=0.9)
model.fit(X_train_norm, y_train_norm, X_valid_norm, y_valid_norm)

ENTRENAMIENTO DEL MODELO NESTEROV ACCELERATED GRADIENT
Época 10/200 - Train Loss: 0.971394 - Valid Loss: 0.867320
Época 20/200 - Train Loss: 0.971394 - Valid Loss: 0.867320
Época 30/200 - Train Loss: 0.971394 - Valid Loss: 0.867320
Época 40/200 - Train Loss: 0.971394 - Valid Loss: 0.867320
Época 50/200 - Train Loss: 0.971394 - Valid Loss: 0.867320
Época 60/200 - Train Loss: 0.971394 - Valid Loss: 0.867320
Época 70/200 - Train Loss: 0.971394 - Valid Loss: 0.867320
Época 80/200 - Train Loss: 0.971394 - Valid Loss: 0.867320
Época 90/200 - Train Loss: 0.971394 - Valid Loss: 0.867320
Época 100/200 - Train Loss: 0.971394 - Valid Loss: 0.867320
Época 110/200 - Train Loss: 0.971394 - Valid Loss: 0.867320
Época 120/200 - Train Loss: 0.971394 - Valid Loss: 0.867320
Época 130/200 - Train Loss: 0.971394 - Valid Loss: 0.867320
Época 140/200 - Train Loss: 0.971394 - Valid Loss: 0.867320
Época 150/200 - Train Loss: 0.971394 - Valid Loss: 0.867320
Época 160/200 - Train Loss: 0.971394 - Valid Loss: 0.8

## 7. Evaluación del Modelo

### 7.1 Evaluación en Train

In [7]:
# Evaluar en conjunto de entrenamiento
print("\n" + "="*60)
print("EVALUACIÓN EN CONJUNTO DE ENTRENAMIENTO")
print("="*60)
train_metrics = model.evaluate(X_train_norm, y_train_norm)
for metric, value in train_metrics.items():
    print(f"{metric}: {value:.6f}")


EVALUACIÓN EN CONJUNTO DE ENTRENAMIENTO
MSE: 0.971394
RMSE: 0.985593
R2: 0.028606


### 7.2 Evaluación en Validación

In [8]:
# Evaluar en conjunto de validación
print("\n" + "="*60)
print("EVALUACIÓN EN CONJUNTO DE VALIDACIÓN")
print("="*60)
valid_metrics = model.evaluate(X_valid_norm, y_valid_norm)
for metric, value in valid_metrics.items():
    print(f"{metric}: {value:.6f}")


EVALUACIÓN EN CONJUNTO DE VALIDACIÓN
MSE: 0.867320
RMSE: 0.931300
R2: -0.070905


### 7.3 Evaluación en Test

In [9]:
# Evaluar en conjunto de prueba
print("\n" + "="*60)
print("EVALUACIÓN EN CONJUNTO DE PRUEBA (TEST)")
print("="*60)
test_metrics = model.evaluate(X_test_norm, y_test_norm)
for metric, value in test_metrics.items():
    print(f"{metric}: {value:.6f}")


EVALUACIÓN EN CONJUNTO DE PRUEBA (TEST)
MSE: 0.751478
RMSE: 0.866878
R2: -0.059379
